<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hand-made-agent" data-toc-modified-id="Hand-made-agent-1">Hand made agent</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Simple-agent" data-toc-modified-id="Simple-agent-1.3">Simple agent</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.3.1">Code</a></span></li><li><span><a href="#Match-visualization" data-toc-modified-id="Match-visualization-1.3.2">Match visualization</a></span></li></ul></li><li><span><a href="#Research-agent" data-toc-modified-id="Research-agent-1.4">Research agent</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.4.1">Code</a></span></li><li><span><a href="#Match-visualization" data-toc-modified-id="Match-visualization-1.4.2">Match visualization</a></span></li></ul></li><li><span><a href="#Build-new-city-tiles" data-toc-modified-id="Build-new-city-tiles-1.5">Build new city tiles</a></span></li><li><span><a href="#Build-new-workers" data-toc-modified-id="Build-new-workers-1.6">Build new workers</a></span></li><li><span><a href="#How-I-would-play-as-a-human?" data-toc-modified-id="How-I-would-play-as-a-human?-1.7">How I would play as a human?</a></span></li><li><span><a href="#Challenges-found-when-creating-an-agent" data-toc-modified-id="Challenges-found-when-creating-an-agent-1.8">Challenges found when creating an agent</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.9">Summary</a></span></li><li><span><a href="#TODO" data-toc-modified-id="TODO-1.10">TODO</a></span></li></ul></li></ul></div>

# Hand made agent

## Goal

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from kaggle_environments import make
from functools import partial

from luxai.utils import render_game_in_html
from luxai.agents.basic import SimpleAgent

Loading environment football failed: No module named 'gfootball'


In [3]:
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game import Game
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game_map import Cell, RESOURCE_TYPES
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.constants import Constants
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game_constants import GAME_CONSTANTS
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux import annotate

In [4]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Simple agent

I will first refactorize the simple agent to ease up the improvement process.

In [5]:
env = make("lux_ai_2021", debug=True, configuration={'width': 12, 'height': 12, 'seed': 0, 'episodeSteps': 361})
initial_info = env.reset()
game_info = env.run(['simple_agent', SimpleAgent()])
render_game_in_html(env)

Opening in existing browser session.


## Research agent

Let's start by improving the simple agent to do research whith the available city tiles. I will first refactorize the simple agent to ease up the improvement process.

### Code

### Match visualization

## Build new city tiles

## Build new workers

## How I would play as a human?

## Challenges found when creating an agent

Let's enumerate the difficulties we have found when trying to create an agent by hand.

## Summary

## TODO

- Move code to the library
- Add tests 
- Function that annotates the resources of the map